In [6]:
import os
import commands
%matplotlib inline
from nifti import *
import nifti
import numpy as np
import matplotlib.pyplot as plt

def print_list(x):
    for i in x:
        print i
def orden(x):
    print x
    #os.system(x)


#  Funciones de preprocesamiento

In [7]:
############# T1_Preproc
def T1_Preproc(iin,fol_out):
    
    iout=fol_out+'highres_reorient.nii.gz'
    comm='fsl5.0-fslreorient2std '+iin+' '+iout+' ; '
    
    #-----------------------------------------
    iin=fol_out+'highres_reorient.nii.gz'
    iout=fol_out+'highres_bet.nii.gz'
    comm=comm+'fsl5.0-bet '+iin+' '+iout+' -m -f 0.3 -g -0.05 ; rm '+iin+' ; '
    
    #------------------------------------------  
    iin=fol_out+'highres_bet.nii.gz'
    iout=fol_out+'highres_bet_5_mm.nii.gz'
    comm=comm+'fsl5.0-fslmaths '+iin+' -kernel gauss 5 -fmean '+iout+' ; rm '+iin
    return comm, iout

In [8]:
############# T2_Preproc
def T2_Preproc(iin,fol_out):
   
    iout_med=fol_out+'rsfMRI_med.nii.gz'
    iout=fol_out+'rsfMRI_01.nii.gz'
    
    comm='fsl5.0-fslmaths '+iin+' -Tmean '+iout_med+' ; '
    comm=comm+'fsl5.0-fslmaths '+iin+' -bptf 20 -1 -add '+iout_med+' '+iout
    comm=comm+' ; rm '+iout_med+' ; rm '+fol_out+'highres_bet_mask.nii.gz'
    return comm, iout

############# T2_Preproc_MCflirt
def T2_Preproc_MC(iin,fol_out):
    iin0=fol_in+'rsfMRI.nii.gz'
    iin=fol_out+'rsfMRI_MC.nii.gz'
    iout_med=fol_out+'rsfMRI_med.nii.gz'
    iout=fol_out+'rsfMRI_MC_01.nii.gz'
    #-----------------------------------------
    comm='fsl5.0-mcflirt -in '+iin0+'  -out '+iin+' -refvol 0 -plots -report ; '
    #-----------------------------------------
    comm=comm+'fsl5.0-fslmaths '+iin+' -Tmean '+iout_med+' ; '
    comm=comm+'fsl5.0-fslmaths '+iin+' -bptf 20 -1 -add '+iout_med+' '+iout+' ; '
    comm=comm+'rm '+iout_med+' ; rm '+iin
    
    return comm, iout

# Motion Correction Conocidos

In [9]:
############# MCflirt
def MC_FSL(iin,fol_out):
    iout=fol_out+'rsfMRI_MC.nii.gz'
    comm='fsl5.0-mcflirt -in '+iin+'  -out '+iout+' -refvol 0 -plots -report'
    return comm, iout

# Funciones de filtro

In [10]:
def Gauss(iin,mm,iout):
    comm='fsl5.0-fslmaths '+iin+' -kernel gauss '+mm+' -fmean '+iout
    return comm

def Aniso(iin,it,iout,fol_out,j):
    place=1
    comm=''
    if(place==1):
        ##usado para servidor
        comm=comm+'(cd ~/abin/; ./3danisosmooth -3D -iters '+it+' -prefix '+iout+' '+iin+')' 
        comm=comm+' ; (cd ~/abin/; ./3dAFNItoNIFTI -prefix '+iout+' '+iout+'+orig)'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+orig.BRIK'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+orig.HEAD'     
    else:
        ##usado por computador 
        comm=comm+'3danisosmooth -3D -iters '+it+' -prefix '+iout+' '+iin
        comm=comm+' ; 3dAFNItoNIFTI -prefix '+iout+' '+iout+'+tlrc'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+tlrc.BRIK'
        comm=comm+' ; rm -R '+fol_out+str(j).zfill(4)+'+tlrc.HEAD'
    return comm

In [11]:
# genera comandos
#[ indicativo_experimento , 1 graduente 0 curvatura , iteraciones, delta_tiempo , conductancia o radio ]

def AnisoITK(typ,iin,iout,niter,dt,conductancia):
    type_filter=['AnisotropicCurvature','AnisotropicGradient'][typ]
    comm='( cd /home/users/aforero/itk_lib/InsightToolkit-4.10.1/Examples/ ; ./'+type_filter+' '+iin+' '+iout+' '+str(niter)+' '+str(dt)+' '+str(conductancia)+' )'
    return comm

# Crear carpeta

In [12]:
def newFol(fol_out):
    comm='mkdir '+fol_out
    return comm

#  Separa Une

In [13]:
def Split(iin,fol_out):
    comm='fsl5.0-fslsplit '+iin+' '+fol_out+' -t'
    return comm    

In [14]:
def merge(list_iin,iout):
    comm='fsl5.0-fslmerge -t '+iout+' '+' '.join(list_iin)
    return comm

# Registro

In [15]:
def Reg(iin,iout,iref,ioutmat,dof):
    comm='fsl5.0-flirt -dof '+dof+' -in '+iin+' -ref '+iref+' -omat '+ioutmat+' -out '+iout
    return comm
    

#  Normalización

In [16]:
def Norm (iin,iout,iatlas,mat_ref):
    comm='fsl5.0-flirt -dof 12 -in '+iin+' -ref '+iatlas+' -applyxfm   -init '+ mat_ref+'  -out '+iout
    return comm



# ICA

In [17]:
def ICASingle(iin,fol_out): 
    comm='fsl5.0-melodic -i '+iin+' -o '+fol_out+' --nobet -d 30 --bgthreshold=10 --tr=2.500 --mmthresh=0.5 --Ostats --Oall'
    return comm

# Seleccion DMN

In [18]:
def SelectDMN(iin='/home/arbey/Escritorio/ICAITK/ImagesC/ICA_Pipeline2_Aniso_3.gica/groupmelodic.ica/melodic_IC.nii.gz',
              fol_iout='/home/arbey/Escritorio/ICAITK/ImagesC/ICA_Pipeline2_Aniso_3.gica/groupmelodic.ica/',
              dir_templ='/home/arbey/Escritorio/ICAITK/Templates/',
              itemplate=0,
              saveImgFormat='/home/arbey/Escritorio/DMN_DUAL_EST4/ImagesC_Aniso1/dr_stage2_subject00000.nii'): 
    
    iiout=fol_iout+'DMN.nii'
    templs=['template_prob.nii','template_bina.nii','template_suma.nii']
    dirtp=dir_templ+templs[itemplate]
    
    comm='python2 DMN.py '+iin+' '+iiout+' '+dirtp+' '+saveImgFormat
    return comm,iiout
#os.system(SelectDMN())

# Test

In [19]:
PSNR

NameError: name 'PSNR' is not defined

# Graficas

In [ ]:
DMN HD

Medias, psnr

In [20]:
def TestIn(ID,testName):
    imgID='C'+str(ID).zfill(3)
    commFolder=newFol('/home/arbey/Escritorio/CPrueba/'+testName)
    fol_in='/home/arbey/Escritorio/CPrueba/ImagesC/'+imgID+'/'
    fol_out='/home/arbey/Escritorio/CPrueba/'+testName+'/'+imgID+'/'

    comm,iiout=T1_Preproc(fol_in,fol_out)
    print commFolder
    print comm

In [21]:
TestIn(1,'filtro_1CN')

mkdir /home/arbey/Escritorio/CPrueba/filtro_1CN
fsl5.0-fslreorient2std /home/arbey/Escritorio/CPrueba/ImagesC/C001/ /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz ; fsl5.0-bet /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz -m -f 0.3 -g -0.05 ; rm /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_reorient.nii.gz ; fsl5.0-fslmaths /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz -kernel gauss 5 -fmean /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet_5_mm.nii.gz ; rm /home/arbey/Escritorio/CPrueba/filtro_1CN/C001/highres_bet.nii.gz


In [22]:
def orden(x):
    print x
    #os.system(x)

# clase de comandos


In [26]:
class Simulacion:
    def __init__(self,ID,folder):
        self._ID=ID
        self._folder=folder
        self._len=120
        self._atlas='/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz'
        self._template='/home/arbey/Escritorio/ICAITK/Templates/'
        self._formatSave='/home/arbey/Escritorio/DMN_DUAL_EST4/ImagesC_Aniso1/dr_stage2_subject00000.nii'
        self._command_list=[]
        
    def getCommandList(self):
        return self._command_list
    
    def FolderInit(self,fol_in):
        print
        print "FolderInit()"
        hr_iin=fol_in+'highres.nii.gz'
        rs_iin=fol_in+'rsfMRI.nii.gz'
        
        orden('mkdir '+self._folder+self._ID)
        return hr_iin, rs_iin

    def Preproc_T1(self,iin):
        orden('mkdir '+self._folder+self._ID+'/Preproc_T1')
        print
        print  "Preproc_T1()"
        
        folderOUT=self._folder+self._ID+'/Preproc_T1/'
        commpreT1,imgout=T1_Preproc(iin,folderOUT)
        self._command_list.append(commpreT1)
        ######## aqui se va a ejecutar
        
        print commpreT1
        return imgout
        
    def Noise_T2(self,img4D,NoiseConfig):
        print
        print "Noise_T2()"
        folderOUT=self._folder+self._ID+'/Noise_T2/'
        orden('mkdir '+self._folder+self._ID+'/Noise_T2')
        
        if(NoiseConfig[0]=='Directo'):
            return img4D
        elif(NoiseConfig[0]=='Noise'):
            print 'Possum'
            ###llama funcion para adicionar ruido y envia configuracion
            img4D=adicionaRuido()
            return img4D

    def Preproc_T2(self,iin):
        print
        print "Preproc_T2()"
        folderOUT=self._folder+self._ID+'/Preproc_T2/'
        orden('mkdir '+self._folder+self._ID+'/Preproc_T2')
        commpreT2,imgout=T2_Preproc(iin,folderOUT)
        self._command_list.append(commpreT2)
        print commpreT2
        return imgout
      

    def MotCorr(self,iin,MCConfig):
        print
        print "MotCorr()"
        folderOUT=self._folder+self._ID+'/MotCorr/'
        orden('mkdir '+self._folder+self._ID+'/MotCorr')
        if(MCConfig[0]=='Directo'):
            print "Directo"
            commSplit=Split(iin,folderOUT)
            self._command_list.append(commSplit)
            print commSplit
            listaImgs=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            print_list(listaImgs)
            return listaImgs
        
        elif(MCConfig[0]=='MCflirt'):
            print "MCflirt"
            commMC,imgOUT=MC_FSL(iin,folderOUT)
            self._command_list.append(commMC)
            print commMC
            commSplit=Split(imgOUT,folderOUT)
            self._command_list.append(commSplit)
            print commSplit
            orden('rm '+imgOUT)
            print commDel
            listaImgs=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            print_list(listaImgs)
            return listaImgs
                     
        elif(MCConfig[0]=='Gauss'):
            print 'Gauss'
            folderOUT_S=folderOUT+'Split/'
            orden('mkdir '+folderOUT_S)
            
            commSplit=Split(iin,folderOUT_S)
            self._command_list.append(commDel)
            listaImgs=[folderOUT_S+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            
            FWHM=str(MCConfig[1])
            for i,iiin in enumerate(listaImgs):
                commGauss=Gauss(iiin,FWHM,folderOUT+str(i).zfill(4)+'.nii.gz')
                self._command_list.append(commGauss)
                print commGauss
            orden('rm '+folderOUT_S)
            listaImgs=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            print_list(listaImgs)
            return listaImgs
       
        elif(MCConfig[0]=='AFNIMotCorr'):
            print "AFNI MotCorr"
            return img4D
        
        elif(MCConfig[0]=='AnisoAFNI'):
            print 'AnisoAFNI'
            folderOUT_S=folderOUT+'Split/'
            orden('mkdir '+folderOUT_S)
            
            commSplit=Split(iin,folderOUT_S)
            self._command_list.append(commSplit)
            listaImgs=[folderOUT_S+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            
            n_it=str(MCConfig[1])
            for i,iiin in enumerate(listaImgs):
                commAnisoAF=Aniso(iiin,n_it,folderOUT+str(i).zfill(4)+'.nii.gz',folderOUT,i)
                self._command_list.append(commAnisoAF)
                print commAnisoAF
                print
            orden('rm '+folderOUT_S)
            listaImgs=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            print_list(listaImgs)
            return listaImgs
        
        elif(MCConfig[0]=='AnisoITK'):
            print 'AnisoITK'   
            
            folderOUT_S=folderOUT+'Split/'
            orden('mkdir '+folderOUT_S)
            
            commSplit=Split(iin,folderOUT_S)
            self._command_list.append(commSplit)
            listaImgs=[folderOUT_S+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            
            n_it=str(MCConfig[2])
            del_t=str(MCConfig[3])
            CorR=str(MCConfig[4])
            
            for i,iiin in enumerate(listaImgs):
                commAnisoITK=AnisoITK(MCConfig[1],iiin,folderOUT+str(i).zfill(4)+'.nii.gz',n_it,del_t,CorR)
                self._command_list.append(commAnisoITK)
                print commAnisoITK
                print
            orden('rm '+folderOUT_S)
            listaImgs=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
            print_list(listaImgs)
            return listaImgs
       
    def Reg(self,iin_Ref,list_Reg,dof):
        print
        print "Reg()"
        folderOUT=self._folder+self._ID+'/Reg/'
        orden('mkdir '+self._folder+self._ID+'/Reg')
        for i,iiin in enumerate(list_Reg):
            commAnatoReg=Reg(iiin,folderOUT+str(i).zfill(4)+'.nii.gz',iin_Ref,folderOUT+str(i).zfill(4),dof)
            self._command_list.append(commAnatoReg)
            print commAnatoReg
            print
        
        dir_erase='/'+'/'.join(list_Reg[i].split('/')[1:-1])
        orden('rm '+dir_erase)
                    
        lis_Img_Reg=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
        print_list(lis_Img_Reg)
        return lis_Img_Reg
        
    def Norm(self,list_Norm,dof): 

        print
        print "Norm()"
        folderOUT=self._folder+self._ID+'/Norm/'
        orden('mkdir '+self._folder+self._ID+'/Norm')
        
        for i,iiin in enumerate(list_Norm):
            if (i==0):
                commAtlasReg=Reg(iiin,folderOUT+str(i).zfill(4)+'.nii.gz',self._atlas,folderOUT+str(i).zfill(4),dof)
                self._command_list.append(commAtlasReg)
                print commAtlasReg
                mat=folderOUT+str(i).zfill(4)+'.mat'
                print mat
            
            else:
                commAtlasReg=Norm(iiin,folderOUT+str(i).zfill(4)+'.nii.gz',self._atlas,mat)
                self._command_list.append(commAtlasReg)
                print commAtlasReg
                print
                
        dir_erase='/'+'/'.join(list_Norm[i].split('/')[1:-1])
        orden('rm '+dir_erase)
        
        dir_NormU='/'+'/'.join(folderOUT.split('/')[1:-1])
        lis_Img_Norm=[folderOUT+str(i).zfill(4)+'.nii.gz' for i in range(self._len)]
        print_list(lis_Img_Norm)
        commMerge=merge(lis_Img_Norm,dir_NormU+'alized.nii.gz')
        self._command_list.append(commMerge)
        print commMerge
        
        dir_erase='/'+'/'.join(lis_Img_Norm[i].split('/')[1:-1])
        orden('rm '+dir_erase)
        
        img_Norm= dir_NormU+'alized.nii.gz'    
        print img_Norm
        return img_Norm

    def Ica(self,desc_4D):
        print
        print "Ica()"
        folderOUT=self._folder+self._ID+'/Ica/'
        orden('mkdir '+self._folder+self._ID+'/Ica')
        commIca=ICASingle(desc_4D,folderOUT)
        self._command_list.append(commIca)
        print commIca
        imgout=folderOUT+'melodic_IC.nii.gz'
        print imgout
        return imgout
        
    def SelectDMN(self,IC_4D):
        print
        print "SelectDMN()"
        folderOUT=self._folder+self._ID+'/SelectDMN/'
        orden('mkdir '+self._folder+self._ID+'/SelectDMN')
        commDMN,DMNOut=SelectDMN(IC_4D,folderOUT,self._template,0,self._formatSave)
        self._command_list.append(commDMN)
        print commDMN
        return DMNOut


In [29]:
def Prueba(IDExperimento,Folder,MCConfig,NoiseConfig,Descripcion):
    sim=Simulacion(IDExperimento,Folder)
    
    hr_iin, rs_iin=sim.FolderInit(Folder)
    
    hr_5mm=sim.Preproc_T1(hr_iin)
    
    imgN=sim.Noise_T2(rs_iin,NoiseConfig)
    
    NameOutP2=sim.Preproc_T2(imgN)
    
    list_MotCorr=sim.MotCorr(NameOutP2,MCConfig)
    
    list_AnatReg=sim.Reg(hr_5mm,list_MotCorr,str(7))
    
    AtlasReg=sim.Norm(list_AnatReg,str(12))
    
    IC=sim.Ica(AtlasReg)
    
    DMN=sim.SelectDMN(IC) 
    
    return sim.getCommandList()
    

In [30]:
listaComm=Prueba('mcflirt_001', #id experimento
       '/home/arbey/Escritorio/CPrueba/ImagesC/C001/', #Folder in
       ['AnisoITK',1,2,0.026,0.01], #Motion correction   --ITK->type(1,0),n_iter,delta_tiempo,conductacia_or_ratio
       ['Directo',0,0,0], #Noise addition
       "Experimento prueba")  # description




FolderInit()
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001
mkdir /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1

Preproc_T1()
fsl5.0-fslreorient2std /home/arbey/Escritorio/CPrueba/ImagesC/C001/highres.nii.gz /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz ; fsl5.0-bet /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz -m -f 0.3 -g -0.05 ; rm /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz ; fsl5.0-fslmaths /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz -kernel gauss 5 -fmean /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet_5_mm.nii.gz ; rm /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz

Noise_T2()
mkdir /home/arbey/Escritorio/CPrueba/I

In [31]:
print_list(listaComm)

fsl5.0-fslreorient2std /home/arbey/Escritorio/CPrueba/ImagesC/C001/highres.nii.gz /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz ; fsl5.0-bet /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz -m -f 0.3 -g -0.05 ; rm /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_reorient.nii.gz ; fsl5.0-fslmaths /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz -kernel gauss 5 -fmean /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet_5_mm.nii.gz ; rm /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T1/highres_bet.nii.gz
fsl5.0-fslmaths /home/arbey/Escritorio/CPrueba/ImagesC/C001/rsfMRI.nii.gz -Tmean /home/arbey/Escritorio/CPrueba/ImagesC/C001/mcflirt_001/Preproc_T2/rsfMRI_med.nii.gz ; fsl5.0-fslmaths /home/arbey/Escritorio/CPrueb